# Development | Implement the different parts and validations of GA

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%matplotlib widget

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import jupyter_black

jupyter_black.load()

# Project
from src.data_connectors import read_input_files
from src.data_connectors.read_input_files import Instance

In [ ]:
instance = 227
instances_path = "../data/input/HRTInstances"
ins_x = read_input_files.read_file(os.path.join(instances_path, f"Instance_{instance}.txt"))

In [ ]:
ins_x.df_setup

In [ ]:
ins_x.df_resources

In [ ]:
ins_x.df_workingspace_resources

In [ ]:
ins_x.df_workingspace_id.head()

## [Flow](https://miro.com/app/board/uXjVPyl00iw=/)

1. Generate **Initial Population**
2. Verify if order of genes is violating the precedence
    1. Discard those that are violating the precedence
3. Duplicate it for all the jobs in the same instance (SAC)
4. Create the array of times (start and end), using the resources time estimator for the respective mode allocation
5. **Fitness Function**: Compute the total time to complete all the jobs in all working spaces of same instance $\Longrightarrow C$ 
6. **Selection**: Retain the genes with lower C (at maximum X)
    1. Discard those that have larger C
    2. Verify if new population has a significant improvement in total C from previous population.
7. **Generate next population**:
    1. **Crossover**: from survival chromosomes, create offsprings 
    2. **Mutation**: add mutation to the created offsprings (initially larger %, and decreases over time).
8. Return to point **2**.

### 0. The chromosome

```python
chromosome = {
    'mode': [0, 1, 2, 2, 1, ... ], # resource ID
    'order': [2, 3, 1, 4, 5, ...], # order of tasks of each position
    }

```

In [ ]:
from src.genetic_algorithm.chromosome import Chromosome

### 1. Generate initial population

By default, it generates population for 1 working space:

**Note**:
- *It is important to generate a large first population, because a lot chromosomes might not survived due to feasibility of solutions.*

In [ ]:
from src.genetic_algorithm import first_population

# Modes
possible_modes = first_population.get_possible_modes(ins_x)

# Tasks
n_tasks = first_population.get_total_number_of_tasks_per_working_space(ins_x)

# Chromosomes
chromosomes = first_population.get_first_population(ins_x, possible_modes, n_tasks)
print(len(chromosomes))

### 2. Verify feasibility of solutions

We need to verify:
- the precedence is valid
- the combination of modes and tasks is possible (the selected mode can perform the associated task) 

In [ ]:
from src.genetic_algorithm import feasibility

In [ ]:
ins_x.df_predecessor_sucessor.head()

In [ ]:
precedence_feasible_chromosomes = []
for i in range(len(chromosomes)):
    chrom = chromosomes[i]
    if feasibility.is_chromosome_precedence_feasible(ins_x, chrom):
        precedence_feasible_chromosomes.append(chrom)

print(f"From {len(chromosomes)} to {len(precedence_feasible_chromosomes)}")

In [ ]:
task_mode_feasible_chromosomes = []
for i in range(len(precedence_feasible_chromosomes)):
    chrom = precedence_feasible_chromosomes[i]
    if feasibility.is_chromosome_task_mode_feasible(ins_x, chrom):
        task_mode_feasible_chromosomes.append(chrom)

print(f"From {len(precedence_feasible_chromosomes)} to {len(task_mode_feasible_chromosomes)}")

### 3. Increment duplication with possible extra working spaces

1. e.g. if other working spaces exist, duplicate the working modes, and duplicate the orders

In [ ]:
from src.genetic_algorithm import replication

In [ ]:
# Generate new orders
chromosome = task_mode_feasible_chromosomes[0]
chromosome

In [ ]:
chromosome_with_replication = replication.update_chromosome_with_replication(ins_x, chromosome)

In [ ]:
chromosome_with_replication

In [ ]:
# for all feasible chromosomes
replicated_chromosomes = []
for chromosome in task_mode_feasible_chromosomes:
    replicated_chromosomes.append(replication.update_chromosome_with_replication(ins_x, chromosome))

len(replicated_chromosomes)

In [ ]:
replicated_chromosomes[:10]

### 4. Get notion of start and end times

1. Go task by task, and get the start and end time, based on allocation of current mode, and based on precedence
2. Compute total timespan - which is the maximum value of endtimes

In [ ]:
from src.genetic_algorithm import time_allocation

In [ ]:
# for all replicated chromosomes
time_allocated_chromosomes = []
for chromosome in replicated_chromosomes:
    time_allocated_chromosomes.append(time_allocation.get_all_time_allocations(ins_x, chromosome))

len(time_allocated_chromosomes)

In [ ]:
# for all results
makespan_all_chromosomes = []
for chromosome_time_allocation in time_allocated_chromosomes:
    makespan_all_chromosomes.append(time_allocation.find_makespan(chromosome_time_allocation))

len(makespan_all_chromosomes)

In [ ]:
min(makespan_all_chromosomes)

In [ ]:
plt.figure()
plt.hist(makespan_all_chromosomes, bins=len(makespan_all_chromosomes))
plt.show()

### 5. Keep fittest chromosomes 

Let's keep the best 100

In [ ]:
from src.genetic_algorithm import fitness

In [ ]:
fittest_chromosomes, fittest_makespan = fitness.keep_fittest_n_chromosomes(
    task_mode_feasible_chromosomes, makespan_all_chromosomes, 100
)
len(fittest_chromosomes)

### 6. Is new population better than previous one?

In [ ]:
# if this is the first population
previous_population_makespan = None
this_population_makespan = fittest_chromosomes

should_we_continue = fitness.is_this_population_better_than_previous_one(
    previous_population_makespan, this_population_makespan
)
print(f"Should we continue iterating? {should_we_continue}")

### 7. Generate next population

with: 
1. Crossover
2. Mutation

In [ ]:
from src.genetic_algorithm import next_population

#### 7.1. Crossover

- **One-Point Crossover**:
    1. select a random crossover point along the length of the chromosomes 
    2. split into 2 proportions on that point, and swapp them between the two parent chromosomes to generate the 2 offsprings.

- **Two-Point Crossover**: similar to one-point crossover, but with two crossover points.

- **Uniform Crossover**: 
    1. Given a probability, select points from each parent and populate for the offsprings.

- **Arithmetic Crossover** is not possible given the current experiment.


In [ ]:
new_generation = next_population.generate_next_population_with_crossover(
    fittest_chromosomes, fittest_makespan
)

In [ ]:
len(new_generation)

#### 7.2. Mutation

Swap mutation: This technique randomly selects two genes within a chromosome and swaps their positions.

Scramble mutation: This technique randomly selects a subset of genes within a chromosome and shuffles their order.

Inversion mutation: This technique randomly selects a subset of genes within a chromosome and reverses their order.

In [ ]:
new_mutated_population = []
probability = 0.4
for chromosome in new_generation:
    new_mutated_population.append(
        next_population.swap_mutation_at_probability(chromosome, probability)
    )